In [1]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox, ttk
from tkcalendar import Calendar
import os
from datetime import datetime
import numpy as np
import re
import time
import folium
from folium.plugins import MarkerCluster
import webbrowser

In [2]:
def convertir_a_decimal(coordenada):
    match = re.match(r"(\d+)°(\d+)'(\d+)\"", coordenada)
    
    if match:
        grados, minutos, segundos = map(float, match.groups())
        decimal = -1*(grados + minutos / 60 + segundos / 3600)
        return round(decimal, 7)
    else:
        print(f"Error en el formato de la coordenada: {coordenada}")
        return None 

In [3]:
empresas=["MultiX", "Blumar", "Yadran", "Australis", "Aquachile", "SalmonesAustral","Camanchaca","Mowi", "Cermaq", "GMT"]
profundidad=[0, 5, 10]

In [4]:
### SELECCIÓN DE FECHA ###
fecha_seleccionada = None
def show_calendar():
    def on_date_select():
        global fecha_seleccionada
        fecha_seleccionada = cal.get_date()
        top.destroy()
        main_window.destroy()

    top = tk.Toplevel(main_window)
    cal = Calendar(top, font="Arial 14", selectmode='day', locale='es_ES')
    cal.pack(fill="both", expand=True)
    ttk.Button(top, text="Seleccionar", command=on_date_select).pack()

main_window = tk.Tk()
main_window.config(width=300, height=200)
main_window.title("Seleccionar Fecha")

button = ttk.Button(text="Seleccionar Fecha", command=show_calendar)
button.place(x=50, y=50)
main_window.mainloop()
fecha= datetime.strptime(fecha_seleccionada, "%d/%m/%y")
fecha=fecha.strftime("%d-%m-%Y")

In [5]:
### BASE DE DATOS ALGAS NOCIVAS ###
conc_path=r"C:\Users\ernes\Documents\Trabajo\Proyecto_Beta\datos_xlsx\Algas_Nocivas.xlsx"
df_algas_nocivas = pd.read_excel(conc_path, sheet_name="CENTRO")

### BASE DE DATOS DE UBICACIONES GEOGRÁFICAS DE CENTROS ###
Centros=[]
centros_path=r"C:\Users\ernes\Documents\Trabajo\Proyecto_Beta\datos_xlsx\Centros.xlsx"
excel_Centros = pd.ExcelFile(centros_path)
for empresa in empresas:
    df_empresa = pd.read_excel(excel_Centros, sheet_name=empresa)
    Centros.append(df_empresa)
    globals()[f"df_{empresa}"] = df_empresa

In [6]:
### BASE DE DATOS CONTABILIZACIONES ####
DB_path=r"C:\Users\ernes\Documents\Trabajo\Proyecto_Beta\datos_xlsx\DB.xlsx"
excel_DB = pd.ExcelFile(DB_path)
Centros_DB=[]
empresas_test=["MultiX", "Blumar"] #Cambiar a empresas cuando haya BD robusta
Especies = []
for empresa in empresas_test:
    df_DB_empresa = pd.read_excel(excel_DB, sheet_name=empresa)
    df_DB_empresa = df_DB_empresa[(df_DB_empresa["MONITOREO"] == "CENTRO") & 
                                  (df_DB_empresa['FECHA'] == fecha) & 
                                  (df_DB_empresa["NOCIVO"] != "NO")]
    especies_empresa = df_DB_empresa["ESPECIE"].unique().tolist()
    Especies.extend(especie for especie in especies_empresa if especie not in Especies)
    Centros_DB.append(df_DB_empresa)
    globals()[f"df_DB_{empresa}"] = df_DB_empresa

In [7]:
lat=Centros[0][Centros[0]['NOMBRE'] == Centros_DB[0]['CENTRO'].iloc[5]]['LATITUDE'].values[0]
print(lat)
print(convertir_a_decimal(lat))
print(type(lat))

41°43'14"
-41.7205556
<class 'str'>


In [8]:
Centros[0][Centros[0]['NOMBRE'] == Centros_DB[0]['CENTRO'].iloc[10]]['LATITUDE'].values[0]

'44°58\'54"'

In [9]:
lat=Centros[1][Centros[1]['NOMBRE']==Centros_DB[1]['CENTRO'].iloc[5]]['LATITUDE'].values[0]
print(lat)
print(convertir_a_decimal(lat))
print(type(lat))

44°19'19"
-44.3219444
<class 'str'>


In [10]:
convertir_a_decimal(Centros[1][Centros[1]['NOMBRE']==Centros_DB[1]['CENTRO'].iloc[5]]['LONGITUDE'].values[0])

-74.3230556

In [11]:
# Crea un mapa centrado
attr = (
    'Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
)
tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
mapa = folium.Map(location=[-44.5378, -74.0256], zoom_start=8, tiles=tiles, attr=attr)

k=0
for centros in Centros_DB:
    size = len(centros['BARRIO'])
    for especie in Especies:
        for prof in profundidad:
            layer_name=f"{especie}_{prof}m_{fecha}"
            current_layer=folium.FeatureGroup(name=layer_name,show=False)
            if especie in df_algas_nocivas['ESPECIE'].to_list():
                valor_normal = df_algas_nocivas.loc[df_algas_nocivas['ESPECIE'] == especie, 'NORMAL'].values[0]
                valor_alerta = df_algas_nocivas.loc[df_algas_nocivas['ESPECIE'] == especie, 'ALERTA'].values[0]
                for i in range(0,size):
                    if (centros['ESPECIE'].iloc[i]==especie) and (centros['PROFUNDIDAD'].iloc[i]==prof):
                        latitude=Centros[k][Centros[k]['NOMBRE'] == centros['CENTRO'].iloc[i]]['LATITUDE'].values[0]
                        longitude=Centros[k][Centros[k]['NOMBRE'] == centros['CENTRO'].iloc[i]]['LONGITUDE'].values[0]
                        location=[convertir_a_decimal(latitude),convertir_a_decimal(longitude)]
                        if centros['CANTIDAD'].iloc[i]<=valor_normal:
                            folium.Circle(location=location, 
                                        color="yellowgreen", fill_color="yellowgreen", 
                                        radius=500, fill_opacity=0.5).add_to(current_layer)                            
                            print('verde')
                        elif centros['CANTIDAD'].iloc[i]>=valor_alerta:
                            folium.Circle(location=location,
                                        color="red", fill_color="red", 
                                        radius=500, fill_opacity=0.5).add_to(current_layer)
                            print('rojo')
                        else:
                            folium.Circle(location=location,
                                        color="yellow", fill_color="yellow",
                                        radius=500, fill_opacity=0.5).add_to(current_layer)
                            print('amarillo')
            current_layer.add_to(mapa)
    k=k+1

folium.LayerControl().add_to(mapa)
mapa_nombre='mapa.html'
mapa.save(mapa_nombre)
webbrowser.open(mapa_nombre)

verde
amarillo
verde
amarillo
amarillo
rojo
rojo
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde
verde


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# Crea un mapa centrado
attr = (
    'Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
)
tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
mapa = folium.Map(location=[-44.5378, -74.0256], zoom_start=8, tiles=tiles, attr=attr)

k=0
    
for especie in Especies:
    for prof in profundidad:
        layer_name=f"{especie}_{prof}m_{fecha}"
        current_layer=folium.FeatureGroup(name=layer_name,show=False)
        if especie in df_algas_nocivas['ESPECIE'].to_list():
            valor_normal = df_algas_nocivas.loc[df_algas_nocivas['ESPECIE'] == especie, 'NORMAL'].values[0]
            valor_alerta = df_algas_nocivas.loc[df_algas_nocivas['ESPECIE'] == especie, 'ALERTA'].values[0]
            for centros in Centros_DB:
                size = len(centros['BARRIO'])
                for i in range(0,size):
                    if (centros['ESPECIE'].iloc[i]==especie) and (centros['PROFUNDIDAD'].iloc[i]==prof):
                        latitude=Centros[k][Centros[k]['NOMBRE'] == centros['CENTRO'].iloc[i]]['LATITUDE'].values[0]
                        longitude=Centros[k][Centros[k]['NOMBRE'] == centros['CENTRO'].iloc[i]]['LONGITUDE'].values[0]
                        location=[convertir_a_decimal(latitude),convertir_a_decimal(longitude)]
                        if centros['CANTIDAD'].iloc[i]<=valor_normal:
                            folium.Circle(location=location, 
                                        color="yellowgreen", fill_color="yellowgreen", 
                                        radius=500, fill_opacity=0.5).add_to(current_layer)                            
                            print('verde')
                        elif centros['CANTIDAD'].iloc[i]>=valor_alerta:
                            folium.Circle(location=location,
                                        color="red", fill_color="red", 
                                        radius=500, fill_opacity=0.5).add_to(current_layer)
                            print('rojo')
                        else:
                            folium.Circle(location=location,
                                        color="yellow", fill_color="yellow",
                                        radius=500, fill_opacity=0.5).add_to(current_layer)
                            print('amarillo')
                k=k+1
        current_layer.add_to(mapa)

folium.LayerControl().add_to(mapa)
mapa_nombre='mapa.html'
mapa.save(mapa_nombre)
webbrowser.open(mapa_nombre)

verde
verde
verde
verde


IndexError: index 0 is out of bounds for axis 0 with size 0